Laboratorio 3

Práctica 3

Autores
Raúl Jiménez Juárez

Beatriz Magán Pinto 

David Alonso Rodríguez Lepina 

Andrés Felipe Sánchez Patarroyo 
         
Asignatura: Aprendizaje estadístico & Data Mining    

Fecha: 31 de octubre de 2019

Objetivo:
El etiquetado de imágenes es una tarea ardua. Es por ello y también debido a sus aplicaciones prácticas que los científicos llevan un tiempo intentando mejorar los métodos para clasificarlas automáticamente. En la aduana del aeropuerto de Madrid se intenta luchar contra el tráfico de animales exóticos. Para ello se va a crear un clasificador que realizando una foto a un animal (en este caso monos) pueda decidir si pertenece a una especie en peligro de extinción o no. Dicho clasificador funcionará mediante un set de entrenamiento donde se buscará un plano que divida las diferentes clases dispuesta en un espacio n-dimensional dependiendo de sus características. 

# Importamos las librerías necesarias

In [1]:
%reset
import numpy as np 
import pandas as pd
import sklearn
import matplotlib as mp
from sklearn import svm
from scipy import misc
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from skimage.color import rgb2grey
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


## 1º Selección de clasificador que más se adapte a nuestro problema de entre los vistos en clase. 
 
Usamos scikit-learn

Utilizamos el clasificador support vector machines para separar el hiperplano de puntos en dos categorías: peligroso o no peligroso, y así obtener si el asteroide en cuestión es peligroso o no para la tierra.

La motivación para elegir support vector machines en nuestro problema son las siguientes:
    - Efectivo en espacio de grandes dimensiones, esto es, en nuestro problema tenemos muchos atributos que estudiar para clasificar el asteroide.
    - Usa un subconjunto de puntos de entrenamiento en la función de decisión, llamado support vector, que mejora la eficiencia en cuanto a memoria. 
    - Versátil: diferentes funciones kernel pueden ser elegidas para la función de decisión.


In [2]:
#Definimos un método para identificar los elementos únicos de una lista. 
def unique(list1): 
    unique_list = [] 
    for x in list1: 
        #Comprueba si el elemento está en la lista de únicos o no. 
        if x not in unique_list: 
            unique_list.append(x) 
    return unique_list #evuelve la lista de elementos únicos

# 1º Cargamos y preprocesamos los datos

In [ ]:
#Definimos el directorio donde se almacenan las imágenes que vamos a preprocesar y posteriormente, a clasificar. 
#Hemos decidido usar el conjunto de validación para que el entrenamiento no se demorase demasiado. 
path = 'Recursos_Lab_3/monos/validation/'

#Creamos una lista para guardar las carpetas, el dataframe de entrenamiento y las etiquetas. 
#En nuestro proyecto, cada carpeta se va a corresponder con un tipo de mono, y por tanto, una etiqueta. 

#Definimos una lista que guarde las etiquetas de los monos generadas durante el entrenamiento. 
etiquetas_monos_entrenamiento = list()
#Definimos una lista que guarda en una sola dimension (flat) las imágenes de los monos. 
flat_features_monkey = list()
# r=root, d=directorios, f = ficheros
for r, d, f in os.walk(path):
    for folder in d:
        for r, d, f in os.walk(path + folder):
            for file in f:
                if '.jpg' in file:
                    #Si el archivo es una imagen con extensión .jpg, lo guardamos en la carpeta. 
                    etiquetas_monos_entrenamiento.append(folder)
                    imagen = Image.open(path + "/" + folder + "/" + file)
                    #Redimensionamos las imágenes para que todas tengan un tamaño de 256x256 con el objetivo de evitar problemas en el procesamiento de las mismas
                    imagen = np.array(imagen.resize((256, 256)))
                    #Redimensionamos las imágenes con el objetivo de guardarlo en una única dimension. filas*columnas*3componentes cada uno
                    array_imagen = np.array(imagen).reshape((len(imagen)*len(imagen[0])*3))
                    flat_features_monkey.append(array_imagen)

In [ ]:
#Guardamos los items unicos de etiquetas 
unique_etiquetas = unique(etiquetas_monos_entrenamiento)
#Creamos un diccionario que guarde las etiquetas de las especies (n0...n9) y sus valores numéricos (0...9) para que el tratamiento
#sea más sencillo. 
etiquetas_mono = {"Especie": unique_etiquetas, 
                     "Value"    : np.arange(0, len(unique_etiquetas))}

#Guardamos en una lista las etiquetas obtenidas. 
etiquetas_n0_n9 = list()
for i in range(len(etiquetas_mono["Value"])):
    for j in range(len(etiquetas_monos_entrenamiento)):
        if etiquetas_mono["Especie"][i] == etiquetas_monos_entrenamiento[j]:
            etiquetas_n0_n9.append(i)

In [ ]:
#Guardamos la lista previamente creada de las características del mono en un array. 
flat_features_monkey = np.array(flat_features_monkey)
print('El tamaño de la matriz de características es : ', flat_features_monkey.shape)

#Definición de Standard Scaler
ss = StandardScaler()
#Ejecutamos el StandardScaler sobre nuestro array flat_features_monkey con el objetivo de normalizar. 
monkeys_stand = ss.fit_transform(flat_features_monkey)

In [ ]:
#Hacemos un split de los datos
X =monkeys_stand
y = np.array(etiquetas_n0_n9)
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=.2, #20% de los datos es para test.
                                                    random_state=1234123)

## 2º Creamos un clasificador que permita saber qué especie de mono es a partir de una imagen. 


In [ ]:
#Realizamos al menos 2 configuraciones y dibujamos una tabla donde se muestre la precisión con la que clasifican. 

#Lineal
#Definición del clasificador de support vector machine.
linear_svm = SVC(kernel='linear', probability=True, random_state=42)
# fit model
linear_svm.fit(X_train, y_train)

#RBF: Kernel de la función radial basis.
rbf_svm = SVC(kernel='rbf', probability=True, random_state=42)
# fit model
rbf_svm.fit(X_train, y_train)

#Realizamos la predicción.
y_pred = linear_svm.predict(X_test[:len(X_test)-6])

#Calculamos la precisión de cada uno de los modelos. 
accuracy = accuracy_score(y_test[:len(y_test)- 6], y_pred)
print('Model lineal accuracy is: ', accuracy)

y_pred = rbf_svm.predict(X_test[:len(X_test)-6])
# calculate accuracy
accuracy = accuracy_score(y_test[:len(y_test)- 6], y_pred)
print('Model rbf accuracy is: ', accuracy)

### El modelo que mejor clasifica  es, por tanto, el modelo lineal porque tiene un accuracy superior al modelo rbf

# 3º Elegimos 5 imágenes de diferentes especies que no hayamos usado ni para entrenar el modelo, ni para evaluarlo. 


In [ ]:
#Clasificamos las 5 imágenes usando el modelo que mejor clasifique de entre los del punto anterior. 
cinco_imagenes = X_test[len(X_test)-6:]
cinco_etiquetas = y_test[len(X_test)-6:]
y_pred_final = linear_svm.predict(cinco_imagenes)

#Calculamos la precisión
accuracy = accuracy_score(cinco_etiquetas, y_pred_final)
print("Accuracy de las 5 imagenes de evaluación es: ", accuracy)

#El error con el que ha funcionado el clasificador es: 
error = 1 - accuracy
print("El error con el que ha funcionado el clasificador es: ", error)